In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Overlaid Histograms

**Please run all cells before this cell, including the import cell at the top of the notebook.**

In [ ]:
temps = Table.read_table('temperatures.csv')
temps

In [ ]:
temps.hist('TMAX', unit='degree')

In [ ]:
temps.select('TMIN', 'TMAX').hist(unit='degree')

## Functions

**Please run all cells before this cell, including the previous example cells and the import cell at the top of the notebook.**

In [ ]:
def double(x):
    """ Double x """
    return 2*x

In [ ]:
double(3)

In [ ]:
double(-4)

In [ ]:
y = 5
double(y/4)

In [ ]:
y

In [ ]:
double(y)

In [ ]:
y

In [ ]:
z = double(y)

In [ ]:
z

In [ ]:
counts = make_array(1, 2, 3)
total = sum(counts)
np.round((counts/total)*100, 2)

In [ ]:
def percents(s):
    """Convert the counts to percents out of the total."""
    total = sum(s)
    return np.round((s/total)*100, 2)

In [ ]:
percents(counts)

In [ ]:
percents(make_array(1, 1, 1, 1))

In [ ]:
counts

In [ ]:
# percents(2)

In [ ]:
# sum(2)

In [ ]:
def percents(counts, decimal_places=2):
    """Convert the counts to percents out of the total."""
    total = sum(counts)
    return np.round((counts/total)*100, decimal_places)

parts = make_array(2, 1, 4)
print("Rounded to 1 decimal place:", percents(parts, 1), "or", percents(parts, decimal_places=1))
print("Rounded to the default number of decimal places:", percents(parts))

## Apply

**Please run all cells before this cell, including the previous example cells and the import cell at the top of the notebook.**

In [ ]:
def cut_off_at_a_billion(x):
    """The smaller of x and 1,000,000,000"""
    return min(x, 1e9)

In [ ]:
cut_off_at_a_billion(12)

In [ ]:
cut_off_at_a_billion(123456)

In [ ]:
cut_off_at_a_billion(1234567890)

In [ ]:
top = Table.read_table('top_movies_2017.csv').where('Studio', 'Fox')
top

In [ ]:
cut_off = top.apply(cut_off_at_a_billion, 'Gross (Adjusted)')
top.with_column('Adjusted but cut', cut_off)

In [ ]:
cut_off_at_a_billion

In [ ]:
type(cut_off_at_a_billion)

In [ ]:
help(cut_off_at_a_billion)

## Prediction

**Please run all cells before this cell, including the previous example cells and the import cell at the top of the notebook.**

In [ ]:
families = Table.read_table('family_heights.csv')
families

In [ ]:
parent_avgs = (families.column('father') + families.column('mother'))/2

In [ ]:
heights = Table().with_columns(
    'Parent Average', parent_avgs,
    'Child', families.column('child'),
    'M/F', families.column('m/f')
)
heights

In [ ]:
heights.scatter('Parent Average', 'Child')

In [ ]:
heights.scatter('Parent Average', 'Child')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2);

In [ ]:
nearby = heights.where('Parent Average', are.between(67.5, 68.5))
nearby_mean = np.average(nearby.column('Child'))
nearby_mean

In [ ]:
heights.scatter('Parent Average', 'Child')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2)
plots.scatter(68, nearby_mean, color='red', s=50);

In [ ]:
def predict(h):
    nearby = heights.where('Parent Average', are.between(h - 1/2, h + 1/2))
    return np.average(nearby.column('Child'))

In [ ]:
predict(68)

In [ ]:
predict(70)

In [ ]:
predict(73)

In [ ]:
predicted_heights = heights.apply(predict, 'Parent Average')

In [ ]:
heights = heights.with_column('Prediction', predicted_heights)

In [ ]:
heights.select('Parent Average', 'Child', 'Prediction').scatter('Parent Average')

## Prediction Accuracy ##

In [ ]:
def difference(x, y):
    return x - y

In [ ]:
pred_errs = heights.apply(difference, 'Prediction', 'Child')
heights = heights.with_column('errors',pred_errs)
heights

In [ ]:
heights.hist('errors')

In [ ]:
heights.hist('errors', group='M/F')

# Discussion Question

In [ ]:
def predict_smarter(h, s):
    nearby = heights.where('Parent Average', are.between(h - 1/2, h + 1/2))
    nearby_same_sex = nearby.where('M/F', s)
    return np.average(nearby_same_sex.column('Child'))

In [ ]:
predict_smarter(68, 'female')

In [ ]:
predict_smarter(68, 'male')

In [ ]:
smarter_predicted_heights = heights.apply(predict_smarter, 'Parent Average', 'M/F')
heights = heights.with_column('Smarter Prediction', smarter_predicted_heights)

In [ ]:
smarter_pred_errs = heights.apply(difference, 'Child', 'Smarter Prediction')
heights = heights.with_column('Smarter Errors', smarter_pred_errs)

In [ ]:
heights.hist('Smarter Errors', group='M/F')